# Train MLModel

In [3]:
import numpy as np
import pandas as pd

StatementMeta(, , , SessionError, )

InvalidHttpRequestToLivy: [TooManyRequestsForCapacity] Unable to submit this request because all the available capacity is currently being used. Cancel a currently running Notebook or Spark Job Definition job, increase your available capacity, or try again later. HTTP status code: 430.

In [ ]:
# Load test data
test = spark.sql("SELECT * FROM Silver.test")
test.show(5)

StatementMeta(, , , Cancelled, )

In [ ]:
display(test, summary=True)

StatementMeta(, , , Cancelled, )

In [ ]:
# Load train data
train = spark.sql("SELECT * FROM Silver.train")
train.show(5)

StatementMeta(, , , Cancelled, )

In [ ]:
display(train, summary=True)

StatementMeta(, , , Cancelled, )

In [ ]:
print((train.count(), len(train.columns)))
print((test.count(), len(test.columns)))

StatementMeta(, , , Cancelled, )

In [ ]:
train = train.toPandas()
test = test.toPandas()
train.head(3)

StatementMeta(, , , Cancelled, )

In [ ]:
print(train.shape)
print(test.shape)

StatementMeta(, , , Cancelled, )

In [ ]:
df1 = test[test.isna().any(axis=1)]
df1

StatementMeta(, , , Cancelled, )

In [ ]:
print(test.shape)
test.dropna(how='any', inplace=True)
print(test.shape)

StatementMeta(, , , Cancelled, )

In [ ]:
df1 = train[train.isna().any(axis=1)]
df1

In [ ]:
y_train = train.SalePrice.values
#train.drop(['SalePrice'], axis=1, inplace=True)
#train.head(3) 

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
def split_dataset(dataset, test_ratio=0.30):
  #test_indices = np.random.rand(len(dataset)) < test_ratio
  #return dataset[~test_indices], dataset[test_indices]
  train_data = dataset[:int(len(dataset) * (1-test_ratio))]
  test_data = dataset[int(len(dataset) * (1-test_ratio)):]
  return train_data, test_data

train_ds_pd, valid_ds_pd = split_dataset(train)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

In [ ]:
!pip install tensorflow_decision_forests -U -qq
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [ ]:
label = 'SalePrice'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label, task = tfdf.keras.Task.REGRESSION)

In [ ]:
tfdf.keras.get_all_models()

StatementMeta(, , , Cancelled, )

In [ ]:
rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
rf.compile(metrics=["mse"]) # Optional, you can use this to include a list of eval metrics

gb = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION)
gb.compile(metrics=["mse"]) # Optional, you can use this to include a list of eval metrics

cart = tfdf.keras.CartModel(task = tfdf.keras.Task.REGRESSION)
cart.compile(metrics=["mse"]) # Optional, you can use this to include a list of eval metrics

StatementMeta(, , , Cancelled, )

In [ ]:
rf.fit(x=train_ds)
gb.fit(x=train_ds)
cart.fit(x=train_ds)

StatementMeta(, , , Cancelled, )

In [ ]:
print(rf.summary())

StatementMeta(, , , Cancelled, )

In [ ]:
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)

StatementMeta(, , , Cancelled, )

In [ ]:
import matplotlib.pyplot as plt
logs = rf.make_inspector().training_logs()
plt.plot([log.num_trees for log in logs], [log.evaluation.rmse for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("RMSE (out-of-bag)")
plt.show()

StatementMeta(, , , Cancelled, )

In [ ]:
evaluation_rf = rf.evaluate(x=valid_ds,return_dict=True)

for name, value in evaluation_rf.items():
  print(f"{name}: {value:.4f}")


StatementMeta(, , , Cancelled, )

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, task = tfdf.keras.Task.REGRESSION)

rf_prediction = rf.predict(x=test_ds)
errors = abs(rf_prediction[:,0] - valid_ds_pd['SalePrice'].values)
MSE=round(np.mean(errors),2)
MAPE=100*(errors/valid_ds_pd['SalePrice'].values)
accuracy=round (100 - np.mean(MAPE),2)
print(f"Rf prediction accuracy: {accuracy}")

gb_prediction = gb.predict(x=test_ds)
errors = abs(gb_prediction[:,0] - valid_ds_pd['SalePrice'].values)
MSE=round(np.mean(errors),2)
MAPE=100*(errors/valid_ds_pd['SalePrice'].values)
accuracy=round (100 - np.mean(MAPE),2)
print(f"Gb prediction accuracy: {accuracy}")

cart_prediction = cart.predict(x=test_ds)
errors = abs(cart_prediction[:,0] - valid_ds_pd['SalePrice'].values)
MSE=round(np.mean(errors),2)
MAPE=100*(errors/valid_ds_pd['SalePrice'].values)
accuracy=round (100 - np.mean(MAPE),2)
print(f"Gb prediction accuracy: {accuracy}")

output_table = valid_ds_pd.copy()
output_table['rf_prediction'] = rf_prediction
output_table['gb_prediction'] = gb_prediction
output_table['cart_prediction'] = cart_prediction
output_table
display(output_table)
output_table = spark.createDataFrame(output_table)
output_table.write.format("delta").mode('overwrite').save("abfss://HousingPrices@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Tables/tfkeras_training")

StatementMeta(, , , Cancelled, )

In [ ]:
np.sqrt(evaluation_rf['mse'])

StatementMeta(, , , Cancelled, )

In [ ]:
inspector = rf.make_inspector()
inspector.evaluation()
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)

StatementMeta(, , , Cancelled, )

In [ ]:
inspector.variable_importances()["NUM_AS_ROOT"]

StatementMeta(, , , Cancelled, )

In [ ]:
plt.figure(figsize=(12, 4))

# Mean decrease in AUC of the class 1 vs the others.
variable_importance_metric = "NUM_AS_ROOT"
variable_importances = inspector.variable_importances()[variable_importance_metric]

# Extract the feature name and importance values.
#
# `variable_importances` is a list of <feature, importance> tuples.
feature_names = [vi[0].name for vi in variable_importances]
feature_importances = [vi[1] for vi in variable_importances]
# The feature are ordered in decreasing importance value.
feature_ranks = range(len(feature_names))

bar = plt.barh(feature_ranks, feature_importances, label=[str(x) for x in feature_ranks])
plt.yticks(feature_ranks, feature_names)
plt.gca().invert_yaxis()

# TODO: Replace with "plt.bar_label()" when available.
# Label each bar with values
for importance, patch in zip(feature_importances, bar.patches):
  plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{importance:.4f}", va="top")

plt.xlabel(variable_importance_metric)
plt.title("NUM AS ROOT of the class 1 vs the others")
plt.tight_layout()
plt.show()

StatementMeta(, , , Cancelled, )

# Predict

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, task = tfdf.keras.Task.REGRESSION)
# RF
rf_preds = rf.predict(test_ds)
# GB
gb_preds = gb.predict(test_ds)
# Cart
cart_preds = cart.predict(test_ds)

StatementMeta(, , , Cancelled, )

In [ ]:
print(f"Number of tests: {test.shape}")
print(f"Number of predictions: {rf_preds.shape}")

predicted_sale_prices_output = test.copy()
predicted_sale_prices_output['PredictedSalePriceRF'] = rf_preds
predicted_sale_prices_output['PredictedSalePriceGB'] = gb_preds
predicted_sale_prices_output['PredictedSalePriceCart'] = cart_preds

In [ ]:
predicted_sale_prices_output

In [ ]:
train_table = spark.createDataFrame(train)
prediction_table = spark.createDataFrame(predicted_sale_prices_output)
train_table.write.format("delta").mode('overwrite').save("abfss://HousingPrices@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Tables/train")
prediction_table.write.format("delta").mode('overwrite').save("abfss://HousingPrices@onelake.dfs.fabric.microsoft.com/Gold.Lakehouse/Tables/tfkeras_prediction")
